In [148]:
using Pkg; Pkg.activate("../.")
using DataFrames
using CSV
using Dates

  Activating environment at `~/tornado_cash_anonymity_tool/Project.toml`


In [149]:
# We load the tornado transactions data
transactions_df = CSV.read("../data/tornado_transactions_truncated.csv", DataFrame)
tornado_addresses = CSV.read("../data/tornadocontracts.csv", DataFrame, header=["Address", "Currency", "3", "4"])[!, "Address"];

In [150]:
# View of first rows of the DataFrame
first(transactions_df, 10)

,hash,nonce,transaction_index
,String,Int64,Int64
1,0xcf97c470a56d96625c7240d3004ae2abd9141d7ffc4383ab6f29a181e3562e8b,4,10
2,0x2fadc4d54bf6e3ec5035f14c0096befe92999866309893e0eeaecdef955c0323,22,118
3,0x9a2584e229b27d3bb87537e4cc4fb7b7cc085f9687742fd5a014bdad2a7f801e,6782,95
4,0x2b40673c95b759b0967ac50185fe5691554819ac7e11ff69fc6972a6e4eb0d50,1748,185
5,0x34478d48b83ce20881013312c6edf130573e561aa0c69c75b09dbb02ba641c97,1963,88
6,0x532078c03eabcffce384d1aae7aa0444b1348eccbdd370b744d9db0206000af4,1276,164
7,0xbf1d3100b2cb759e43aadd8777b08696144918b6402aaa4f3d850d470cc67490,2482,102
8,0x3dc8617176f45c0f33a95589c202b22f9ba8aa7067bb92887517e20b237907e3,789,196
9,0x3199144ba98021e523c5ec1e4f4dcf64446d69b825da7251d2d2a0a794038d0d,9,190


In [151]:
tornado_addresses

19-element Vector{String}:
 "0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3"
 "0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144"
 "0x07687e702b410fa43f4cb4af7fa097918ffd2730"
 "0x23773e65ed146a459791799d01336db287f25334"
 "0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc"
 "0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936"
 "0x910cbd523d972eb0a6f4cae4618ad62622b39dbf"
 "0xa160cdab225685da1d56aa342ad8841c3b53f291"
 "0xd96f2b1c14db8458374d9aca76e26c3d18364307"
 "0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d"
 "0x169ad27a470d064dede56a2d3ff727986b15d52b"
 "0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f"
 "0x178169b423a011fff22b9e3f3abea13414ddd0f1"
 "0x610b717796ad172b316836ac95a2ffad065ceab4"
 "0xbb93e510bbcd0b7beb5a853875f9ec60275cf498"
 "0x22aaa7720ddd5388a3c0a3333430953c68f1849b"
 "0x03893a7c7463ae47d46bc7f091665f1893656003"
 "0x2717c5e28cf931547b621a5dddb772ab6a35b701"
 "0xd21be7248e0197ee08e0c20d4a96debdac3d20af"

In [152]:
names(transactions_df)

21-element Vector{String}:
 "hash"
 "nonce"
 "transaction_index"
 "from_address"
 "to_address"
 "value"
 "gas"
 "gas_price"
 "input"
 "receipt_cumulative_gas_used"
 "receipt_gas_used"
 "receipt_contract_address"
 "receipt_root"
 "receipt_status"
 "block_timestamp"
 "block_number"
 "block_hash"
 "max_fee_per_gas"
 "max_priority_fee_per_gas"
 "transaction_type"
 "receipt_effective_gas_price"

In [153]:
# A DataFrame with only the addresses data of transactions.
address_pairs = select(transactions_df, :from_address, :to_address);

In [154]:
first(address_pairs, 10)

,from_address,to_address
,String,String
1,0xb050dec5a9010f8b77a3962369b7bc737d3ed4a5,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d
2,0x6e92bc493c6abbdd6a1b18416f003de2c873ab50,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d
3,0xbe4d1e137a24af091be80ae58d652279665e3a27,0x722122df12d4e14e13ac3b6895a86e84145b6967
4,0x20bb3095a4852f4c97d7a188e9f7183c85acfc49,0x722122df12d4e14e13ac3b6895a86e84145b6967
5,0xdd36d585a0d32b4ae6f1790bc4292308e21a42e3,0x722122df12d4e14e13ac3b6895a86e84145b6967
6,0x28a51fd7b39b49ad4d7813d89e3d4444d79e3ac6,0x722122df12d4e14e13ac3b6895a86e84145b6967
7,0x49136693081f2c18e2cf14428dd78cd90a22dc1f,0x722122df12d4e14e13ac3b6895a86e84145b6967
8,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0xa160cdab225685da1d56aa342ad8841c3b53f291
9,0xb77562124be8ac967cf7fc24573fe252aa39d95d,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d


In [187]:
# Receives a DataFrame with the transactions and an array with the tornado addresses.
# Returns an array with the compromised addresses

function same_addresses(transactions_df, tornado_addresses)
    unique_deposit_addresses = filter(addr -> addr ∉ tornado_addresses, unique(transactions_df[!,"from_address"]))
    unique_withdrawal_addresses = filter(addr -> addr ∉ tornado_addresses, unique(transactions_df[!,"to_address"]))
    compromised_addresses = []
    
    for address in unique_withdrawal_addresses
        if address in unique_deposit_addresses
            push!(compromised_addresses, address)
            println("The address $address is compromised!")
        end
    end
    compromised_addresses
end

same_addresses (generic function with 1 method)

In [156]:
same_addresses(transactions_df, tornado_addr)

Any[]

In [195]:
# Receives a DataFrame with the transactions and an array with the tornado addresses.
# Returns an array with the compromised addresses and timestamps of the withdrawal.

function same_addresses_(transactions_df, tornado_addresses)
    deposit_transactions = filter(row -> row["from_address"] ∉ tornado_addresses, transactions_df)
    withdrawal_transactions = filter(row -> row["from_address"] ∈ tornado_addresses, transactions_df)
    
    withdraw_addresses_and_timestamps = []
    
    for row in eachrow(withdrawal_transactions)
        if row["to_address"] in deposit_transactions[!, "to_address"]
            push!(withdraw_addresses_and_timestamps, (row.from_address, row.block_timestamp))
            println("Dangerous transaction in $(row.block_timestamp)")
        end
    end
    withdraw_addresses_and_timestamps
end

same_addresses_ (generic function with 1 method)

In [158]:
same_addresses_(transactions_df, tornado_addresses)

Any[]

Since there are no matching contracts in our data, I create a fake DataFrame to test the functions:

In [192]:
test_df = DataFrame(["address1" "address2" Dates.DateTime("18/05/20010 16:12", "dd/mm/yyyy HH:MM");
           "address2" "address3" Dates.DateTime("18/05/20011 16:12", "dd/mm/yyyy HH:MM");
           "address4" "address1" Dates.DateTime("18/05/20013 16:12", "dd/mm/yyyy HH:MM")], ["from_address", "to_address", "block_timestamp"])

,from_address,to_address,block_timestamp
,Any,Any,Any
1,address1,address2,20010-05-18T16:12:00
2,address2,address3,20011-05-18T16:12:00
3,address4,address1,20013-05-18T16:12:00


In [178]:
test_tornado_addresses = ["address2", "address4"]

2-element Vector{String}:
 "address2"
 "address4"

In [189]:
same_addresses(test_df, test_tornado_addresses)

The address address1 is compromised!


1-element Vector{Any}:
 "address1"

In [196]:
same_addresses_(test_df, test_tornado_addresses)

Any[]